# Characteristic
A point cloud is a set of points with coordinates, and different from an image and meshes. Point positions in a point cloud depend on coordinates. Moreover, a point cloud does not have a specified relationship between points. Therefore, a point cloud is unstructured data. In this section, we introduce characteristics of point cloud representation in processing.

This section introduces the following characteristics of point cloud:
- Properties of points
- Sparse or dense points
- Definition of Neighborhood
- Invariance to a rigid transformation
- unordered points


## Properties of points
In addition to coordinates, points can have colors and normals, etc:
- **Color (RGB)**: the color is used when displaying points on viewers, or for point cloud processing.
- **Normals**: we can get the surface of a 3D object or detect planes from normals.
- **Intensity**:  "intensity" is the laser reflection intensity, and depends on the reflectivity of the object surface that the laser hits. Intensity can be used for feature extraction.


## Sparse or dense points

ここで言う疎と密は、ある範囲内の点の密度のことを指します。この密度についてわかりやすい例として、センサーから取得した点群が当てはまります。実際のセンサーから取得したデータを下のコードの実行で確認できます。


上の出力はセンサーから点までの距離を色の変化によって示しています。赤がセンサーから近く、徐々に青になることでセンサーから距離が離れていることを示しています。ここで、センサーからの距離によって点がまばらになっていくことがわかると思います。
これは、距離が遠くなるにつれて、オクルージョンやセンサー自身の解像度などの複数の要因が影響してまばらなになっています(この要因についてはここでは詳細に説明しません)。

## Definition of Neighborhood
この隣接関係が明確でない特性は、問題を生むことがあります。例えば、隣接する要素を必要とする畳み込みアルゴリズムの適用を阻むという問題があります。これらの問題を解決するには以下のような方法が挙げられます。

1. 点群の各点に対してk近傍(kNN)やボールクエリ等を実施し、点の近くにある点との隣接関係を持たせる。
2. 点群をボクセル表現に変換する。

1番目の方法は、点の周りの情報を収集するためによく使われます。例えば、handcrafted featureを生成する際は、点の周りにある他の点の情報を参照するためにkNNが使用されます。また、深層学習手法などの畳込み機構を設けた処理では、同様に周りの点の特徴を畳み込むために使用されます。
2番目の方法は、点群を画像の様なグリッドに沿ったデータフォーマットに変換することができます。これにより、例えば深層学習では画像用の2D CNNを拡張した3D CNNの適用が可能になりました。


## 剛体変換に対する不変性 (タスクによる)
点群に対して剛体変換を用いることで点群が持つ形状を崩さずに姿勢の変更や平行移動が可能です。タスクによっては剛体変換された点群に対する処理結果が変化しないようにする必要があります。例えばオブジェクトに対する分類タスクでは、与えられる入力点群の姿勢が一定ではない可能性があり、一定でない場合はどの様な姿勢に対しても不変な予測を行う必要があります。
「タスクによる」とあるのは、姿勢や位置そのものを予測するタスク(例:姿勢推定)などが当てはまらないためです。これらのタスクは、与えられたデータの姿勢や位置に合わせて予測を行う必要があります。





## 点の順不同による形状不変性

点群は画像や文字列の様に要素に順序があるわけではないため、点の入力順序を変更したとしても点群が示す形状は変わりません。例えば、オブジェクト点群内にある点の順序をランダムに入れ替えたとしても、オブジェクトの形状は変わりません。
例として、オリジナルの点群と点の順序を入れ替えた点群をそれぞれ表示します。この表示で使うパッケージは以下のとおりです。

点の順序入れ替え前と後を表示するコードは以下のとおりです。X軸で0~1の間にあるウサギの点群が入れ替え前、1~2の間にあるウサギの点群が入れ替え後です。  
Note: 見比べを行うために入れ替え後の点群の位置を平行移動させています。  
問題点: 平行移動による形状が変わらないことへの言及がない

上の出力から見てわかるように、ウサギの形状も色も変わってないことが示されていると思います。  
実際にこれらの点群を処理する場合、その手法の出力は点群の点の順序に依存しない処理方法が必要となります。例えば機械学習の分野の分類タスクを解く場合、ある点群AとAの点の順序を変えた点群Bを機械学習モデルに入力したとしても、AとBの分類予測ラベルは同じでなければいけません。

# Reference

